In [8]:
import simulation_commander as simc
import grSim_Commands_pb2
import grSim_Packet_pb2
import time
import numpy as np
import math

client = simc.Client(ip='127.0.0.1', port=20011)

In [9]:
def prep_message_and_send(yellowTeam, id, kickspeedx, kickspeedz, veltangent, velnormal, velangular, spinner, wheelsspeed, wheel1, wheel2, wheel3, wheel4):
    packet = grSim_Packet_pb2.grSim_Packet()
    packet.commands.isteamyellow = yellowTeam
    packet.commands.timestamp = 0
    robot_command = grSim_Commands_pb2.grSim_Robot_Command()
    robot_command.id = id
    robot_command.kickspeedx = kickspeedx
    robot_command.kickspeedz = kickspeedz
    robot_command.veltangent = veltangent
    robot_command.velnormal = velnormal
    robot_command.velangular = velangular
    robot_command.spinner = spinner
    robot_command.wheelsspeed = wheelsspeed
    robot_command.wheel1 = wheel1   # front left
    robot_command.wheel2 = wheel2   # back left
    robot_command.wheel3 = wheel3   # back right
    robot_command.wheel4 = wheel4   # front right
    packet.commands.robot_commands.append(robot_command)
    
    while True:
        client.send(packet)
        time.sleep(0.01)

In [10]:
R = 0.09

r=0.027

######## Angulos das rodas em relação ao eixo x
######## 0 -> 30 graus front right
######## 1 -> 150 graus front left
######## 2 -> 225 graus back left
######## 3 -> 315 graus back right
wheel_alngles = [math.pi/6, (math.pi*5)/6, (math.pi*5)/4, (math.pi*7)/4]

######## Matriz Jacobiana
jacobian = np.matrix([[math.sin(wheel_alngles[0]), math.cos(wheel_alngles[0]), R],
                      [math.sin(wheel_alngles[1]), math.cos(wheel_alngles[1]), R],
                      [math.sin(wheel_alngles[2]), math.cos(wheel_alngles[2]), R],
                      [math.sin(wheel_alngles[3]), math.cos(wheel_alngles[3]), R],])

######## Eixo x, eixo y, rotação
velocities = np.matrix([[1], [1], [0]])

In [ ]:
wheels_acc = (1/r*jacobian*velocities).T.tolist()[0]
wheels_acc

In [12]:
import grSim_Replacement_pb2

packet = grSim_Packet_pb2.grSim_Packet()
robot_replacement = grSim_Replacement_pb2.grSim_RobotReplacement()
robot_replacement.x = 0
robot_replacement.y = 3
robot_replacement.dir = 270
robot_replacement.id = 0
robot_replacement.yellowteam = 1
packet.replacement.robots.append(robot_replacement)


client.send(packet)
time.sleep(0.01)

In [ ]:
prep_message_and_send(True, 0, 0, 0, 0, 0, 0, 0, 1, wheels_acc[1], wheels_acc[2], wheels_acc[3], wheels_acc[0])

In [ ]:
from move import *

test_bang_bang_optimal()

In [ ]:
from move import *

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time
 
start_time = time.time()
############################################
pos_x = -4500
pos_y = 3000.0
vel_x = -10.0
vel_y = 0.0

goal_pos_x = 4500
goal_pos_y = -3000
goal_vel_x = 0
goal_vel_y = 0.0

x_init = (pos_x, pos_y, vel_x, vel_y)
x_goal = (goal_pos_x, goal_pos_y, goal_vel_x, goal_vel_y)

steer_start_time = time.time()
control_path = time_optimal_steer_2d(x_init, x_goal)
steer_comp_time = time.time() - steer_start_time

control_time = control_time(control_path)

time_unit = 1

path_position_x = [pos_x]
path_position_y = [pos_y]
path_velocity_x = [vel_x]
path_velocity_y = [vel_y]
acc_x = []
acc_y = []

discrete_time = np.linspace(0, control_time, num=int(control_time/time_unit))
print(control_path)
for t in discrete_time:
    sub_path = []
    total_time = 0
    for session in control_path:
            if session[1] + total_time < t:
                sub_path.append(session)
                total_time += session[1]
            else:
                sub_path.append([session[0], t - total_time])
                break

    current_state = integrate_control_2d((pos_x, pos_y, vel_x, vel_y), sub_path)
    
    path_position_x.append(current_state[0])
    path_position_y.append(current_state[1])
    path_velocity_x.append(current_state[2])
    path_velocity_y.append(current_state[3])
    acc_x.append(sub_path[-1][0][0])
    acc_y.append(sub_path[-1][0][1])

# for control_session in control_path:
#     for dt in np.arange(0, control_session[1], time_unit):
#         vx = path_velocity_x[-1] + control_session[0][0] * time_unit
#         vy = path_velocity_y[-1] + control_session[0][1] * time_unit

#         x = path_position_x[-1] + vx * time_unit
#         y = path_position_y[-1] + vy * time_unit

#         path_position_x.append(x)
#         path_position_y.append(y)
#         path_velocity_x.append(vx)
#         path_velocity_y.append(vy)
#         acc_x.append(control_session[0][0])
#         acc_y.append(control_session[0][1])

final_time = time.time() - start_time
############################################


fig = plt.figure(figsize=(8, 12))
gs = gridspec.GridSpec(2, 3, width_ratios=[2, 2, 6], height_ratios=[1, 1])

velocity_x_plot = plt.subplot(gs[0, 0])
velocity_y_plot = plt.subplot(gs[1, 0])
acc_x_plot = plt.subplot(gs[0, 1])
acc_y_plot = plt.subplot(gs[1, 1])
position_plot = plt.subplot(gs[:, 2])

X_axis = np.arange(0, control_time, time_unit)

velocity_x_plot.plot(X_axis, path_velocity_x[0:len(X_axis)], color='black')
velocity_x_plot.set_xlabel('Time')
velocity_x_plot.set_ylabel('Velocity X')
velocity_x_plot.grid()

velocity_y_plot.plot(X_axis, path_velocity_y[0:len(X_axis)], color='black')
velocity_y_plot.set_xlabel('Time')
velocity_y_plot.set_ylabel('Velocity Y')
velocity_y_plot.grid()

acc_x_plot.plot(X_axis[0:len(acc_x)], acc_x, color='black')
acc_x_plot.set_xlabel('Time')
acc_x_plot.set_ylabel('Acceleration X')
acc_x_plot.grid()

acc_y_plot.plot(X_axis[0:len(acc_x)], acc_y, color='black')
acc_y_plot.set_xlabel('Time')
acc_y_plot.set_ylabel('Acceleration Y')
acc_y_plot.grid()

position_plot.axis([-4500, 4500, -3000, 3000])

position_plot.plot(path_position_x[0:len(X_axis)], path_position_y[0:len(X_axis)], color='black')
position_plot.set_xlabel('Position X')
position_plot.set_ylabel('Position Y')
position_plot.scatter(pos_x, pos_y, marker=',', color='green')
position_plot.annotate('Initial', (pos_x, pos_y))
position_plot.scatter(goal_pos_x, goal_pos_y, marker='x', color='red')
position_plot.annotate('Goal', (goal_pos_x, goal_pos_y))
position_plot.grid()

# obstacle_plt = plt.Circle((-0, -0), 90, color='red')
# padding_plt = plt.Circle((-0, -0), 180, color='red', fill=False)
# position_plot.add_patch(obstacle_plt)
# position_plot.add_patch(padding_plt)

# obstacle1 = StaticRobotObstacle(-0, -0, 90)
x, y = [], []
check_time = 1
index = 0
points_checked = []
while check_time <= control_time:
    sub_path = []
    total_time = 0
    for session in control_path:
            if session[1] + total_time < check_time:
                sub_path.append(session)
                total_time += session[1]
            else:
                sub_path.append([session[0], check_time - total_time])
                break
    # ponto
    check_state = integrate_control_2d((pos_x, pos_y, vel_x, vel_y), sub_path)
    # is there a collision
    # collision = obstacle1.is_collision(np.array([check_state[0], check_state[1]]), 90)
    # if collision:
    #     print(f'collision on {check_state[0]} {check_state[1]}')
    # # time off set
    # distance = obstacle1.calc_distance(np.array([check_state[0], check_state[1]]))
    # # print(distance, dynamic_step_over_path(distance))
    # check_time += dynamic_step_over_path(distance, control_time, 1)

    # points_checked.append([[check_state[0], check_state[1]], collision])
    # x.append(check_state[0])
    # y.append(check_state[1])

print(f'Points checked for colision {len(points_checked)}')

position_plot.scatter(x, y, marker='.', color='blue')

plt.title('Rest to Rest Bang Bang Trajectory')
plt.show()

print(f'Control Path: {control_path}')
print(f'Control Time: {control_time}')
print(f'Computing Control time: {steer_comp_time}')
print(f'Whole Program Time: {final_time}')
